### Tree species classification example
This notebook gives an example of using a convolutional neural network to classify tree species in the Sierra Nevada forest.

First we download the NEON data and label files from our dataset stored on Zenodo.

In [1]:
import os
import sys
import tqdm
import argparse

from wget import download

from experiment.paths import *

# make output directory if necessary
if not os.path.exists('data'):
    os.makedirs('data')

files = [ 'Labels_Trimmed_Selective.CPG',
          'Labels_Trimmed_Selective.dbf',
          'Labels_Trimmed_Selective.prj',
          'Labels_Trimmed_Selective.sbn',
          'Labels_Trimmed_Selective.sbx',
          'Labels_Trimmed_Selective.shp',
          'Labels_Trimmed_Selective.shp.xml',
          'Labels_Trimmed_Selective.shx',
          'NEON_D17_TEAK_DP1_20170627_181333_reflectance.tif',
          'NEON_D17_TEAK_DP1_20170627_181333_reflectance.tif.aux.xml',
          'NEON_D17_TEAK_DP1_20170627_181333_reflectance.tif.enp',
          'NEON_D17_TEAK_DP1_20170627_181333_reflectance.tif.ovr',
          'D17_CHM_all.tfw',
          'D17_CHM_all.tif',
          'D17_CHM_all.tif.aux.xml',
          'D17_CHM_all.tif.ovr',
        ]

for f in files:
    if not os.path.exists('data/%s'%f):
        print('downloading %s'%f)
        download('https://zenodo.org/record/3468720/files/%s?download=1'%f,'data/%s'%f)
        print('')

Next we loads and co-register our data sources, including the hyperspectral image, the canopy height model, and the tree labels.  Then we build a dataset of patches and their corresponding labels and store it in a HDF5 file for easy use in Keras.

In [2]:
import numpy as np
import tqdm
from experiment.paths import *
import os

from canopy.vector_utils import *
from canopy.extract import *
import h5py as h5

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

# Load the metadata from the image.
with rasterio.open(image_uri) as src:
    image_meta = src.meta.copy()

os.makedirs('example',exist_ok=True)

seed = 0

# Load the shapefile and transform it to the hypersectral image's CRS.
polygons, labels = load_and_transform_shapefile(labels_shp_uri,'SP',image_meta['crs'])

# Cluster polygons for use in stratified sampling
centroids = np.stack([np.mean(np.array(poly['coordinates'][0]),axis=0) for poly in polygons])
cluster_ids = KMeans(10).fit_predict(centroids)
rasterize_shapefile(polygons, cluster_ids, image_meta, 'example/clusters.tiff')
stratify = cluster_ids

# alternative: stratify by species label
# stratify = labels

# Split up polygons into train, val, test here
train_inds, test_inds = train_test_split(range(len(polygons)),test_size=0.1,random_state=seed,stratify=stratify)

# Save ids of train,val,test polygons
with open('example/' + train_ids_uri,'w') as f:
    f.writelines(["%d\n"%ind for ind in train_inds])
with open('example/' + test_ids_uri,'w') as f:
    f.writelines(["%d\n"%ind for ind in test_inds])

# Separate out polygons
train_polygons = [polygons[ind] for ind in train_inds]
train_labels = [labels[ind] for ind in train_inds]
test_polygons = [polygons[ind] for ind in test_inds]
test_labels = [labels[ind] for ind in test_inds]

# Rasterize the shapefile to a TIFF.  Using LZW compression, the resulting file is pretty small.
train_labels_raster = rasterize_shapefile(train_polygons, train_labels, image_meta, 'example/' + train_labels_uri)
test_labels_raster = rasterize_shapefile(test_polygons, test_labels, image_meta, 'example/' + test_labels_uri)

# Extract patches and labels
patch_radius = 7
height_threshold = 5
train_image_patches, train_patch_labels = extract_patches(image_uri,patch_radius,chm_uri,height_threshold,'example/' + train_labels_uri)
test_image_patches, test_patch_labels = extract_patches(image_uri,patch_radius,chm_uri,height_threshold,'example/' + test_labels_uri)


100%|██████████| 1909/1909 [00:39<00:00, 48.41it/s]


Now we set up and train the convolutional neural network model.

In [6]:
import numpy as np
import h5py as h5
from tqdm import tqdm, trange
import os
import sys

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD, Adam

from sklearn.decomposition import PCA
from joblib import dump, load
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

from canopy.model import PatchClassifier
from experiment.paths import *

from tensorflow.keras import backend as K
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

np.random.seed(0)
tf.set_random_seed(0)

out = 'example'
lr = 0.0001
epochs = 20

x_all = train_image_patches
y_all = train_patch_labels

class_weights = compute_class_weight('balanced',range(8),y_all)
print('class weights: ',class_weights)
class_weight_dict = {}
for i in range(8):
    class_weight_dict[i] = class_weights[i]

def estimate_pca():
    x_samples = x_all[:,7,7]
    pca = PCA(32,whiten=True)
    pca.fit(x_samples)
    return pca

"""Normalize training data"""
pca = estimate_pca()
dump(pca,out + '/pca.joblib')

x_shape = x_all.shape[1:]
x_dtype = x_all.dtype
y_shape = y_all.shape[1:]
y_dtype = y_all.dtype
x_shape = x_shape[:-1] + (pca.n_components_,)

print(x_shape, x_dtype)
print(y_shape, y_dtype)

classifier = PatchClassifier(num_classes=8)
model = classifier.get_patch_model(x_shape)

print(model.summary())

model.compile(optimizer=SGD(lr,momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

def apply_pca(x):
    N,H,W,C = x.shape
    x = np.reshape(x,(-1,C))
    x = pca.transform(x)
    x = np.reshape(x,(-1,H,W,x.shape[-1]))
    return x

checkpoint = ModelCheckpoint(filepath=out + '/' + weights_uri, monitor='val_acc', verbose=True, save_best_only=True, save_weights_only=True)
reducelr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

x_all = apply_pca(x_all)

def augment_images(x,y):
    x_aug = []
    y_aug = []
    with tqdm(total=len(x)*8,desc='augmenting images') as pbar:
        for rot in range(4):
            for flip in range(2):
                for patch,label in zip(x,y):
                    patch = np.rot90(patch,rot)
                    if flip:
                        patch = np.flip(patch,axis=0)
                        patch = np.flip(patch,axis=1)
                    x_aug.append(patch)
                    y_aug.append(label)
                    pbar.update(1)
    return np.stack(x_aug,axis=0), np.stack(y_aug,axis=0)

x_all, y_all = augment_images(x_all,y_all)

train_inds, val_inds = train_test_split(range(len(x_all)),test_size=0.1,random_state=0)
x_train = np.stack([x_all[i] for i in train_inds],axis=0)
y_train = np.stack([y_all[i] for i in train_inds],axis=0)
x_val = np.stack([x_all[i] for i in val_inds],axis=0)
y_val = np.stack([y_all[i] for i in val_inds],axis=0)

batch_size = 32

model.fit( x_train, y_train,
           epochs=epochs,
           batch_size=batch_size,
           validation_data=(x_val,y_val),
           verbose=1,
           callbacks=[checkpoint,reducelr],
           class_weight=class_weight_dict)




class weights:  [ 0.74829501  2.29405615  1.21758085  0.48317187  0.7970631  24.93668831
  2.45540281  0.61169959]
(15, 15, 32) int16
() uint8
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 13, 13, 32)        9248      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 9, 9, 128)         73856     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 7, 7, 128)         147584    
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 5, 5, 128)         147584    

augmenting images: 100%|██████████| 122888/122888 [00:01<00:00, 73026.99it/s]


Train on 110599 samples, validate on 12289 samples
Epoch 1/20
110599/110599 [==============================] - 41s 366us/step - loss: 1.7037 - acc: 0.5477 - val_loss: 1.1374 - val_acc: 0.8593

Epoch 00001: val_acc improved from -inf to 0.85931, saving model to example/weights.hdf5
Epoch 2/20
110599/110599 [==============================] - 44s 399us/step - loss: 0.9412 - acc: 0.9016 - val_loss: 0.9102 - val_acc: 0.9249

Epoch 00002: val_acc improved from 0.85931 to 0.92489, saving model to example/weights.hdf5
Epoch 3/20
110599/110599 [==============================] - 44s 397us/step - loss: 0.8229 - acc: 0.9410 - val_loss: 0.8023 - val_acc: 0.9571

Epoch 00003: val_acc improved from 0.92489 to 0.95712, saving model to example/weights.hdf5
Epoch 4/20
110599/110599 [==============================] - 44s 395us/step - loss: 0.7664 - acc: 0.9590 - val_loss: 0.7577 - val_acc: 0.9675

Epoch 00004: val_acc improved from 0.95712 to 0.96745, saving model to example/weights.hdf5
Epoch 5/20
11059

Now we run the trained model on the full image in tiles.

In [7]:
import numpy as np
import cv2
from math import floor, ceil
import tqdm
from joblib import dump, load

import rasterio
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio.vrt import WarpedVRT

from canopy.model import PatchClassifier
from experiment.paths import *

from tensorflow.keras import backend as K
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

pca = load(out + '/pca.joblib')

# "no data value" for labels
label_ndv = 255

# radius of square patch (side of patch = 2*radius+1)
patch_radius = 7

# height threshold for CHM -- pixels at or below this height will be discarded
height_threshold = 5

# tile size for processing
tile_size = 128

# tile size with padding
padded_tile_size = tile_size + 2*patch_radius

# open the hyperspectral or RGB image
image = rasterio.open(image_uri)
image_meta = image.meta.copy()
image_ndv = image.meta['nodata']
image_width = image.meta['width']
image_height = image.meta['height']
image_channels = image.meta['count']

# load model
input_shape = (padded_tile_size,padded_tile_size,pca.n_components_)
tree_classifier = PatchClassifier(num_classes=8)
training_model = tree_classifier.get_patch_model(input_shape)
training_model.load_weights(out + '/' + weights_uri)
model = tree_classifier.get_convolutional_model(input_shape)

# calculate number of tiles
num_tiles_y = ceil(image_height / float(tile_size))
num_tiles_x = ceil(image_width / float(tile_size))

print('Metadata for image')
for key in image_meta.keys():
    print('%s:'%key)
    print(image_meta[key])
    print()

# create predicted label raster
predict_meta = image_meta.copy()
predict_meta['dtype'] = 'uint8'
predict_meta['nodata'] = label_ndv
predict_meta['count'] = 1
predict = rasterio.open(out + '/' + predict_uri, 'w', compress='lzw', **predict_meta)

# open the CHM
chm = rasterio.open(chm_uri)
chm_vrt = WarpedVRT(chm, crs=image.meta['crs'], transform=image.meta['transform'], width=image.meta['width'], height=image.meta['height'],
                   resampling=Resampling.bilinear)

# dilation kernel
kernel = np.ones((patch_radius*2+1,patch_radius*2+1),dtype=np.uint8)

def apply_pca(x):
    N,H,W,C = x.shape
    x = np.reshape(x,(-1,C))
    x = pca.transform(x)
    x = np.reshape(x,(-1,H,W,x.shape[-1]))
    return x

# go through all tiles of input image
# run convolutional model on tile
# write labels to output label raster
with tqdm.tqdm(total=num_tiles_y*num_tiles_x) as pbar:
    for y in range(patch_radius,image_height-patch_radius,tile_size):
        for x in range(patch_radius,image_width-patch_radius,tile_size):
            pbar.update(1)

            window = Window(x-patch_radius,y-patch_radius,padded_tile_size,padded_tile_size)

            # get tile from chm
            chm_tile = chm_vrt.read(1,window=window)
            if chm_tile.shape[0] != padded_tile_size or chm_tile.shape[1] != padded_tile_size:
                pad = ((0,padded_tile_size-chm_tile.shape[0]),(0,padded_tile_size-chm_tile.shape[1]))
                chm_tile = np.pad(chm_tile,pad,mode='constant',constant_values=0)
          
            chm_tile = np.expand_dims(chm_tile,axis=0)
            chm_bad = chm_tile <= height_threshold

            # get tile from image
            image_tile = image.read(window=window)
            image_pad_y = padded_tile_size-image_tile.shape[1]
            image_pad_x = padded_tile_size-image_tile.shape[2]
            output_window = Window(x,y,tile_size-image_pad_x,tile_size-image_pad_y)
            if image_tile.shape[1] != padded_tile_size or image_tile.shape[2] != padded_tile_size:
                pad = ((0,0),(0,image_pad_y),(0,image_pad_x))
                image_tile = np.pad(image_tile,pad,mode='constant',constant_values=-1)

            # re-order image tile to have height,width,channels
            image_tile = np.transpose(image_tile,axes=[1,2,0])

            # add batch axis
            image_tile = np.expand_dims(image_tile,axis=0)
            image_bad = np.any(image_tile < 0,axis=-1)

            image_tile = image_tile.astype('float32')
            image_tile = apply_pca(image_tile)
            
            # run tile through network
            predict_tile = np.argmax(model.predict(image_tile),axis=-1).astype('uint8')

            # dilate mask
            image_bad = cv2.dilate(image_bad.astype('uint8'),kernel).astype('bool')

            # set bad pixels to NDV
            predict_tile[chm_bad[:,patch_radius:-patch_radius,patch_radius:-patch_radius]] = label_ndv
            predict_tile[image_bad[:,patch_radius:-patch_radius,patch_radius:-patch_radius]] = label_ndv

            # undo padding
            if image_pad_y > 0:
                predict_tile = predict_tile[:,:-image_pad_y,:]
            if image_pad_x > 0:
                predict_tile = predict_tile[:,:,:-image_pad_x]

            # write to file
            predict.write(predict_tile,window=output_window)

image.close()
chm.close()
predict.close()

  0%|          | 0/774 [00:00<?, ?it/s]

Metadata for image
nodata:
None

transform:
| 1.00, 0.00, 319344.00|
| 0.00,-1.00, 4101691.00|
| 0.00, 0.00, 1.00|

width:
1028

count:
426

height:
10948

dtype:
int16

crs:
+init=epsg:32611

driver:
GTiff



 89%|████████▉ | 688/774 [02:58<00:20,  4.23it/s]


Finally we run an analysis of the classification performance on the test set.

In [8]:
import numpy as np

import rasterio
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio.vrt import WarpedVRT
from rasterio.mask import mask

from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.geometry import mapping

import tqdm

from math import floor, ceil

from experiment.paths import *

from canopy.vector_utils import *
from canopy.extract import *

import sklearn.metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

train_inds = np.loadtxt(out + '/' + train_ids_uri,dtype='int32')
test_inds = np.loadtxt(out + '/' + test_ids_uri,dtype='int32')

# Load the metadata from the image.
with rasterio.open(image_uri) as src:
    image_meta = src.meta.copy()

# Load the shapefile and transform it to the hypersectral image's CRS.
polygons, labels = load_and_transform_shapefile(labels_shp_uri,'SP',image_meta['crs'])

train_labels = [labels[ind] for ind in train_inds]
test_labels = [labels[ind] for ind in test_inds]

# open predicted label raster
predict = rasterio.open(out + '/' + predict_uri)
predict_raster = predict.read(1)
ndv = predict.meta['nodata']

def get_predictions(inds):
    preds = []
    for ind in inds:
        poly = [mapping(Polygon(polygons[ind]['coordinates'][0]))]
        out_image, out_transform = mask(predict, poly, crop=False)
        out_image = out_image[0]
        
        label = labels[ind]

        rows, cols = np.where(out_image != ndv)
        predict_labels = []
        for row, col in zip(rows,cols):
            predict_labels.append(predict_raster[row,col])
        predict_labels = np.array(predict_labels)
        
        hist = [np.count_nonzero(predict_labels==i) for i in range(8)]
        majority_label = np.argmax(hist)
        preds.append(majority_label)
    return preds

def calculate_confusion_matrix(labels,preds):
    mat = np.zeros((8,8),dtype='int32')
    for label,pred in zip(labels,preds):
        mat[label,pred] += 1
    return mat

def calculate_fscore(labels,preds):
    return sklearn.metrics.f1_score(labels,preds,average='micro')

test_preds = get_predictions(test_inds)
 
report = classification_report(test_labels, test_preds)
mat = confusion_matrix(test_labels,test_preds)
print('classification report:')
print(report)
print('confusion matrix:')
print(mat)

classification report:
             precision    recall  f1-score   support

          0       0.62      0.89      0.73         9
          1       0.00      0.00      0.00         1
          2       0.82      1.00      0.90         9
          3       1.00      0.88      0.93        16
          4       0.88      1.00      0.93         7
          5       0.00      0.00      0.00         2
          6       0.56      0.71      0.63         7
          7       1.00      0.67      0.80        21

avg / total       0.83      0.79      0.80        72

confusion matrix:
[[ 8  1  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  0  0  0]
 [ 1  0  0 14  1  0  0  0]
 [ 0  0  0  0  7  0  0  0]
 [ 0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  2  5  0]
 [ 3  0  2  0  0  0  2 14]]
